In [21]:
import traceback
import sys
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests

In [22]:
# Websites to scrape
nasa_mars_news_url = "https://mars.nasa.gov/news/"
jpl_mars_space_images_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
mars_facts_url = "https://space-facts.com/mars/"
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [29]:
class ChromeBrowser(object): 
    """
    """
    def __init__(self, url):
        self.url = url 
      
    def __enter__(self): 
        executable_path = {'executable_path': ChromeDriverManager().install()}
        self.browser = Browser('chrome', **executable_path, headless=False)
        self.browser.visit(self.url)
        return self.browser
  
    def __exit__(self, exc_type, exc_val, exc_tb):
        if (exc_type is not None):
            traceback.print_exception(exc_type, exc_val, exc_tb)
        if (self.browser is not None):
            self.browser.quit() 
        return True

In [17]:
# Scrape the NASA Mars News Site and collect the latest News Title 
# and Paragraph Text. Assign the 
# text to variables that you can 
# reference later.
news_title = ""
news_p = ""

with ChromeBrowser(nasa_mars_news_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    news_title = soup.find('div', class_='content_title').text
    news_p = soup.find('div', class_='article_teaser_body').text
    
print(news_title)
print()
print(news_p)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Bootcamp\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




Mars Now

NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


In [34]:
# Use splinter to navigate the site and find the image url 
# for the current Featured Mars Image and assign the url 
# string to a variable called `featured_image_url`.

# Make sure to find the image url to the full size `.jpg` image.

# Make sure to save a complete url string for this image.

featured_image_url = ""

with ChromeBrowser(jpl_mars_space_images_url) as browser: 
    html = browser.html
    soup = bs(html, "html.parser")
    header_imgs = soup.find(class_='header').find_all("img")
    for img in header_imgs:
        if (img["src"] and "featured" in img["src"].lower()):
            featured_image_url = jpl_mars_space_images_url.\
            replace("index.html", img["src"]) 
    
print(featured_image_url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Bootcamp\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


In [ ]:
# Visit the Mars Facts webpage and use Pandas to scrape the table
# containing facts about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.

mars_facts

In [ ]:
# Visit the USGS Astrogeology site to obtain high resolution images
# for each of Mar's hemispheres.

# You will need to click each of the links to the hemispheres in 
# order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere 
# image, and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys `img_url`
# and `title`.

# Append the dictionary with the image url string and the hemisphere 
# title to a list. This list will contain one dictionary for each
# hemisphere.

hemisphere_image_urls = []